## Predicting alcohol consumption based on demographic factors

A big data assignment, testing the feasibility of predicting alcohol use.
Data used is from [Riksmaten 2010-11](https://www.livsmedelsverket.se/matvanor-halsa--miljo/matvanor---undersokningar/riksmaten-2010-11---vuxna) by Swedish food agency.

More on the data format [here](https://www.livsmedelsverket.se/om-oss/psidata/apimatvanor).

Column of interest is 'Alko' representing mean daily intake of alcohol in grams for observed inviduals.

#### Import libraries

In [ ]:
import urllib3
import xml.etree.ElementTree as ET
import hashlib
import pandas
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from graphviz import Source
from sklearn import tree
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt

#### Step 1. Download survey data

In [ ]:
http = urllib3.PoolManager()

for i in range(1,19):
    print(f'Currently downloading: {i}.xml')
    r = http.request('GET', f'http://www7.slv.se/apilivsmedel/undersokningservice.svc/undersokning/14/{i}', preload_content=False)

    with open(f'{i}.xml', 'wb') as out:
        while True:
            data = r.read(32)
            if not data:
                break
            out.write(data)

    r.release_conn()

print('Done.')

#### Step 2. Process xml

In [ ]:
tree = ET.parse('1.xml')
root = tree.getroot()
deltagarlista = root.find('DeltagareLista')
tree._setroot(deltagarlista)
root = tree.getroot()

for i in range(2,19):
    print(f'Currently processing: {i}.xml')
    nexttree = ET.parse(f'{i}.xml')
    nextroot = nexttree.getroot()
    nextdeltagarlista = nextroot.find('DeltagareLista')
    nexttree._setroot(nextdeltagarlista)
    nextroot = nexttree.getroot()
    for nextdeltagare in nextroot.findall('Deltagare'):
        root.append(nextdeltagare)

print('Cleaning up and writing xml to file.')

for deltagare in root.findall('Deltagare'):
    deltagare.remove(deltagare.find('RegistreringsDagar'))
    alkohol = deltagare.find('NaringsvardenPerDag').find('Alko')
    deltagare.append(alkohol)
    deltagare.remove(deltagare.find('NaringsvardenPerDag'))

tree.write('output.xml')
print('Done.')

#### Step 3. Compare xml (optional)

In [ ]:
m = hashlib.sha256()

with open('output.xml', mode='rb') as file:
    m.update(file.read())

if m.hexdigest() == '02577d008dd232ae24c5e4d57d3caba6a5ccaa42bee4f2ef31829111d596584b':
    print('Success, survey data has not been changed. 👍')
else:
    print('Error, looks like the survey data has been changed. Results might differ!')

#### Step 4. Filter data

In [ ]:
survey = pandas.read_xml("output.xml")
survey

Filter out rows where education level is missing.

In [ ]:
survey = survey[survey['HogstaUtb'] != "Uppgift saknas  (här ingår även personer äldre än 74 år)"]

Scatterplot for age and registered alcohol use.

In [ ]:
survey.plot.scatter(x='Alder',y='Alko',c='Black')

In [ ]:
survey.boxplot(column='Alko')

Filter observations beyond 2.7σ from the mean.

In [ ]:
q3 = survey['Alko'].quantile(.75)
q1 = survey['Alko'].quantile(.25)
iqr = q3 - q1
outliers = (q3 + 1.5 * iqr)

survey = survey[survey['Alko'] <= outliers] #Filter on IQR times 1.5

Perform mean imputation on some missing values considered missing at random.

In [ ]:
survey['DeltagarensInkomst'] = survey['DeltagarensInkomst'].fillna(survey['DeltagarensInkomst'].mean())
survey['HushalletsInkomst'] = survey['HushalletsInkomst'].fillna(survey['HushalletsInkomst'].mean())
survey['Langd'] = survey['Langd'].fillna(survey['Langd'].mean())
survey['Vikt'] = survey['Vikt'].fillna(survey['Vikt'].mean())

Calculate if observed consumption is more than ["low risk"](https://www.omsystembolaget.se/folkhalsa/kropp-och-halsa/bruk-och-beroende/hjalp-och-stod/ofarligt-drickande/).

In [ ]:
survey['Risky'] = survey['Alko'].apply(lambda x: 1 if x>17.1 else 0)

Create dummy variables for sex and University education (or similar).

In [ ]:
survey = pandas.get_dummies(survey, columns=['Kon'], drop_first=True)
def went_to_uni(education):
    if education == "Forskarutbildning" or education == "Eftergymnasial utbildning 3 år eller längre (exkl. forskarutbildning)":
        return 1
    else:
        return 0

survey["Uni"] = survey["HogstaUtb"].apply(went_to_uni)

Print correlation matrix of our data so far.

In [ ]:
survey.corr()

We can't see any useful correlation yet, albeit this is only linear ones, lets try training some models anyway.

##### DecisionTreeRegressor predicting average alcohol intake based on sex, age, income.

In [ ]:
alko_data = survey.Alko  # save this for training later

survey_alko = survey[['Kon_Man', 'Alder', 'DeltagarensInkomst']] # Use most promising columns

X_train, X_test, y_train, y_test = train_test_split(survey_alko, alko_data, test_size=0.25)
print("Our training data has {} rows".format(len(X_train)))
print("Our test data has {} rows".format(len(X_test)))

In [ ]:
regressor = DecisionTreeRegressor(max_depth=3)
regressor_fit = regressor.fit(X_train.values, y_train.values)

dt_scores = cross_val_score(regressor_fit, X_train, y_train, cv = 5)
print("mean cross validation score: {}".format(np.mean(dt_scores)))
print("score without cross validation: {}".format(regressor_fit.score(X_train, y_train)))

In [ ]:
sample = X_test.head(10)
sample['Alko'] = regressor.predict(sample)
sample

##### DecisionTreeClassifier predicting non-"low risk" drinking based on sex, age, income.

In [ ]:
risk_data = survey.Risky  # save this for training later

survey_risk = survey[['Kon_Man', 'Alder', 'DeltagarensInkomst']] # Use most promising columns

X_train, X_test, y_train, y_test = train_test_split(survey_risk, risk_data, test_size=0.25)
print("Our training data has {} rows".format(len(X_train)))
print("Our test data has {} rows".format(len(X_test)))

In [ ]:
classifier = DecisionTreeClassifier(max_depth=3)
classifier_fit = classifier.fit(X_train.values, y_train.values)

dt_scores = cross_val_score(classifier_fit, X_train, y_train, cv = 5)
print("mean cross validation score: {}".format(np.mean(dt_scores)))
print("score without cross validation: {}".format(classifier_fit.score(X_train, y_train)))

These validation scores are way too high as tree is biased due to one class dominating.

Verdict: Not very useful, do we need more data?, better data?